In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('data\phase1_results.csv')

In [15]:
# prepare dataframe for plotting (recompute if necessary)
df_plot = df.copy()

# Plot a horizontal bar chart showing the 'Category' basis the total count of 'ASIN' in each category using plotly
df_barplot = df_plot.groupby('Category').agg(ASIN_count=('ASIN', 'count')).reset_index().copy()
df_barplot = df_barplot.sort_values(by='ASIN_count', ascending=False)
fig_bar = px.bar(df_barplot, 
                 x='ASIN_count', 
                 y='Category', 
                 orientation='h',
                 labels={'ASIN_count': 'Number of Products', 'Category': 'Category'},
                 title='Count of Products by Category')

# Reverse the y-axis to have the highest count on top
fig_bar.update_yaxes(autorange='reversed')

# Set custom font for the chart (Space Grotesk, fallback to Arial if not available)
fig_bar.update_layout(
    font=dict(
        family='Tahoma, Arial, sans-serif',
        size=14,
        color='black'
    ),
    title_font=dict(
        family='Tahoma, Arial Black, Arial, sans-serif',
        size=20,
        color='darkblue'
    ),
    xaxis_title_font=dict(
        family='Tahoma, Arial',
        size=16,
        color='black'
    ),
    yaxis_title_font=dict(
        family='Tahoma, Arial',
        size=16,
        color='black'
    )
)
fig_bar.show()

In [11]:
# Show the 'categories' of the products with the maximum number of total ratings as list in the 'Glb_Ratings_Rev' column using plotly; the figure should also be horizontal bar chart and sort the category in descending order basis the total ratings
df_ratings = df_plot.groupby('Category').agg(Total_Ratings=('Glb_Ratings_Rev', 'sum')).reset_index().copy()
df_ratings = df_ratings.sort_values(by='Total_Ratings', ascending=False)
fig_ratings = px.bar(df_ratings, 
                     x='Total_Ratings', 
                     y='Category', 
                     orientation='h',
                     labels={'Total_Ratings': 'Total Ratings', 'Category': 'Category'},
                     title='Total Ratings by Category')

# Update the layout to use Tahoma font as shared above and apply consistent styling
fig_ratings.update_traces(marker_color='#636efa',
                          textposition='auto',
                          hovertemplate='Total Ratings=%{x}<br>Category=%{y}<extra></extra>')

fig_ratings.update_layout(
    font=dict(
        family='Tahoma, Arial, sans-serif',
        size=14,
        color='black'
    ),
    title_font=dict(
        family='Tahoma, Arial Black, Arial, sans-serif',
        size=20,
        color='darkblue'
    ),
    xaxis_title_font=dict(
        family='Tahoma, Arial',
        size=16,
        color='black'
    ),
    yaxis_title_font=dict(
        family='Tahoma, Arial',
        size=16,
        color='black'
    ),
    margin=dict(t=60)
)

# Reverse y-axis so the highest total ratings appear at the top (we already sorted descending)
fig_ratings.update_yaxes(autorange='reversed')

fig_ratings.show()

In [21]:
# # Pricing distribution across different categories using a normal distribution curve in plotly; use data in the 'df' dataframe including 'Category' and 'Price_Nos'
# # Create a density plot for price distribution across categories using the 'df' dataframe

# import plotly.figure_factory as ff

# # Prepare data: use top N categories to keep the plot readable
# top_n = 6
# price_col = 'Price_Nos'
# cat_col = 'Category'

# df_prices = df[[cat_col, price_col]].dropna(subset=[price_col]).copy()

# top_cats = df[cat_col].value_counts().nlargest(top_n).index.tolist()

# # collect price arrays per category
# dist_data = [df_prices[df_prices[cat_col] == cat][price_col].values for cat in top_cats]

# # create distribution plot (KDE only, no histogram bars) using plotly.figure_factory
# fig_price_dist = ff.create_distplot(
#     dist_data,
#     top_cats,
#     show_hist=False,      # Hide histogram bars
#     show_rug=False,
#     colors=px.colors.qualitative.Plotly
# )

# fig_price_dist.update_layout(
#     title_text=f'Price Distribution by Category (Top {len(top_cats)} Categories)',
#     xaxis_title='Price (USD)',
#     yaxis_title='Density',
#     font=dict(family='Tahoma, Arial, sans-serif', size=14, color='black'),
#     title_font=dict(family='Tahoma, Arial Black, Arial, sans-serif', size=20, color='darkblue')
# )

# fig_price_dist.show()
# # ...existing code...
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Select top 9 categories for the grid
top_n = 6
price_col = 'Price_Nos'
cat_col = 'Category'
df_prices = df[[cat_col, price_col]].dropna(subset=[price_col]).copy()
top_cats = df[cat_col].value_counts().nlargest(top_n).index.tolist()

# Create 3x3 subplot grid
fig = make_subplots(rows=2, cols=3, subplot_titles=top_cats)

for idx, cat in enumerate(top_cats):
    row = idx // 3 + 1
    col = idx % 3 + 1
    cat_prices = df_prices[df_prices[cat_col] == cat][price_col]
    fig.add_trace(
        go.Histogram(
            x=cat_prices,
            name=cat,
            marker_color=px.colors.qualitative.Plotly[idx % len(px.colors.qualitative.Plotly)],
            showlegend=False
        ),
        row=row, col=col
    )

fig.update_layout(
    height=900, width=1200,
    title_text="Price Distribution by Category (Top 9 Categories)",
    font=dict(family='Tahoma, Arial, sans-serif', size=14, color='black'),
    title_font=dict(family='Tahoma, Arial Black, Arial, sans-serif', size=20, color='darkblue')
)

# Optional: adjust axes for all subplots
fig.update_xaxes(title_text="Price (USD)")
fig.update_yaxes(title_text="Count")

fig.show()

In [25]:
# Standardize prices (z-score) for the top 6 categories and plot KDEs filled with slight transparency
import plotly.figure_factory as ff

# Prepare standardized data per category
standardized_data = []
labels = []
colors = px.colors.qualitative.Plotly[:len(top_cats)]

for cat in top_cats:
    vals = df_prices[df_prices[cat_col] == cat][price_col].dropna()
    if len(vals) < 2:
        # skip categories without enough data to compute a meaningful std
        continue
    z = (vals - vals.mean()) / vals.std(ddof=0)  # z-score normalization
    standardized_data.append(z.values)
    labels.append(cat)

# Create KDE plot (no histograms) using plotly.figure_factory
fig_kde = ff.create_distplot(
    standardized_data,
    labels,
    show_hist=False,
    show_rug=False,
    colors=colors
)

# After fig_kde is created
kde_trace_idx = 0
for trace in fig_kde.data:
    # KDE traces are 'scatter' with fill='tozeroy'
    if getattr(trace, "fill", None) == "tozeroy":
        trace.name = labels[kde_trace_idx]
        kde_trace_idx += 1

# Fill under curves and set slight transparency so overlaps are visible; ensure legend visible
fig_kde.update_traces(fill='tozeroy', opacity=0.45, hoverinfo='x+y', showlegend=True)

# Layout and axis labels
fig_kde.update_layout(
    title_text='Standardized Price Distribution by Category (Top 6 Categories) — z-score',
    xaxis_title='Standardized Price (z-score)',
    yaxis_title='Density',
    font=dict(family='Tahoma, Arial, sans-serif', size=14, color='black'),
    title_font=dict(family='Tahoma, Arial Black, Arial, sans-serif', size=20, color='darkblue'),
    legend_title_text='Category'
)

fig_kde.show()
